In [2]:
#imports
import numpy as np
import pandas as pd
import os
import re
import string
import time
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

retiveing data from the folders 

In [119]:
#set directory locations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
wine_processed_folder = parent_directory + '/data/wine-com/processed/'
wunderground_raw_folder = parent_directory + '/data/wunderground-com/raw/'
wunderground_processed_folder = parent_directory + '/data/wunderground-com/processed/'

In [120]:
df = pd.read_csv(wunderground_processed_folder + '1678665697.3855995.txt',
          sep = '|')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/georgegarcia/Documents/GitHub/Wine-Rating-Projections-Using-Historic-Weather-Patterns/notebooks/data/wunderground-com/processed/1678665697.3855995.txt'

In [3]:
df = pd.read_csv('/Users/georgegarcia/Documents/GitHub/Wine-Rating-Projections-Using-Historic-Weather-Patterns/data/wunderground-com/processed/1678665697.3855994.txt',
                        sep = '|')

In [4]:
df.shape

(1115, 456)

In [5]:
print(df)

                                           Product_Name Product_Varietal  \
0     Domaine Taupenot-Merme Nuits-Saint-Georges Les...       Pinot Noir   
1      Domaine de la Romanee-Conti Echezeaux Grand Cru        Pinot Noir   
2                Lincourt Rancho Santa Rosa Pinot Noir        Pinot Noir   
3               Domaine Claude Dugat Gevrey-Chambertin        Pinot Noir   
4     Domaine Prieur-Brunet Santenay Maladiere Premi...       Pinot Noir   
...                                                 ...              ...   
1110   R. Lopez de Heredia Rioja Vina Tondonia Reserva       Tempranillo   
1111           Villota Vina Gena Vinedo Singular Rioja       Tempranillo   
1112                                 Cune Gran Reserva       Tempranillo   
1113                             Bosco Barolo La Serra          Nebbiolo   
1114                          Masseto (. Liter Magnum)            Merlot   

                                         Product_Origin Product_Price  \
0     Nuits-St

In [6]:
# drop columns with "min" or "max" in the name
cols_to_drop = [col for col in df.columns if 'min' in col.lower() or 'max' in col.lower()]
df_filitered = df.drop(cols_to_drop, axis=1)

print(df_filitered)

                                           Product_Name Product_Varietal  \
0     Domaine Taupenot-Merme Nuits-Saint-Georges Les...       Pinot Noir   
1      Domaine de la Romanee-Conti Echezeaux Grand Cru        Pinot Noir   
2                Lincourt Rancho Santa Rosa Pinot Noir        Pinot Noir   
3               Domaine Claude Dugat Gevrey-Chambertin        Pinot Noir   
4     Domaine Prieur-Brunet Santenay Maladiere Premi...       Pinot Noir   
...                                                 ...              ...   
1110   R. Lopez de Heredia Rioja Vina Tondonia Reserva       Tempranillo   
1111           Villota Vina Gena Vinedo Singular Rioja       Tempranillo   
1112                                 Cune Gran Reserva       Tempranillo   
1113                             Bosco Barolo La Serra          Nebbiolo   
1114                          Masseto (. Liter Magnum)            Merlot   

                                         Product_Origin Product_Price  \
0     Nuits-St

In [7]:
df_filitered.shape

(1115, 216)

In [8]:
# Select only the numerical columns for outlier analysis
num_df = df_filitered.select_dtypes(include=np.number)

# Detect and remove outliers
Q1 = num_df.quantile(0.25)
Q3 = num_df.quantile(0.75)
IQR = Q3 - Q1
num_df_out = num_df[~((num_df < (Q1 - 1.5 * IQR)) | (num_df > (Q3 + 1.5 * IQR))).any(axis=1)]

# Merge the numerical dataframe with the original dataframe to include the text columns
df_out = pd.merge(num_df_out, df_filitered, left_index=True, right_index=True)

In [9]:
print(df_out)

      User_Avg_Rating_x  User_Rating_Count_x  Product_Vintage_x  \
0                   0.0                    0               2017   
3                   0.0                    0               2020   
4                   0.0                    0               2020   
6                   0.0                    0               2020   
7                   0.0                    0               2016   
...                 ...                  ...                ...   
950                 0.0                    0               2018   
970                 3.1                    7               2018   
994                 4.0                   11               2018   
1010                0.0                    0               2020   
1013                0.0                    0               2018   

      Critical_Avg_Rating_x  Critical_Rating_Count_x  Appellation_Level_x  \
0                 93.000000                        3                    5   
3                 92.000000              

In [10]:
# Get the column names
column_names = df_filitered.columns.tolist()

# Print the column names
print(column_names)

['Product_Name', 'Product_Varietal', 'Product_Origin', 'Product_Price', 'Product_Attributes', 'User_Avg_Rating', 'User_Rating_Count', 'Product_Vintage', 'Critical_Avg_Rating', 'Critical_Rating_Count', 'Appellation_Level', 'Weather_Origin', 'Aug_Avg_High_Temperature', 'Aug_Avg_Temperature', 'Aug_Avg_Low_Temperature', 'Aug_Avg_High_ Dew_Point', 'Aug_Avg_ Dew_Point', 'Aug_Avg_Low_ Dew_Point', 'Aug_Avg_High_Humidity', 'Aug_Avg_Humidity', 'Aug_Avg_Low_Humidity', 'Aug_Avg_High_Wind_Speed ', 'Aug_Avg_Wind_Speed ', 'Aug_Avg_Low_Wind_Speed ', 'Aug_Avg_High_Pressure', 'Aug_Avg_Pressure', 'Aug_Avg_Low_Pressure', 'Aug_Average_Precipitation', 'Aug_Total_Precipitation', 'Nov_Avg_High_Temperature', 'Nov_Avg_Temperature', 'Nov_Avg_Low_Temperature', 'Nov_Avg_High_ Dew_Point', 'Nov_Avg_ Dew_Point', 'Nov_Avg_Low_ Dew_Point', 'Nov_Avg_High_Humidity', 'Nov_Avg_Humidity', 'Nov_Avg_Low_Humidity', 'Nov_Avg_High_Wind_Speed ', 'Nov_Avg_Wind_Speed ', 'Nov_Avg_Low_Wind_Speed ', 'Nov_Avg_High_Pressure', 'Nov_Avg_P

In [11]:
text_columns = ['Product_Name', 'Product_Varietal', 'Product_Origin', 'Product_Price', 'Product_Attributes'] 
numeric_columns = ['User_Avg_Rating', 'User_Rating_Count', 'Product_Vintage', 'Critical_Avg_Rating', 'Critical_Rating_Count',
             'Appellation_Level', 'Weather_Origin', 'Aug_Avg_High_Temperature', 'Aug_Avg_Temperature', 'Aug_Avg_Low_Temperature', 
             'Aug_Avg_High_ Dew_Point', 'Aug_Avg_ Dew_Point', 'Aug_Avg_Low_ Dew_Point', 'Aug_Avg_High_Humidity', 'Aug_Avg_Humidity', 
             'Aug_Avg_Low_Humidity', 'Aug_Avg_High_Wind_Speed ', 'Aug_Avg_Wind_Speed ', 'Aug_Avg_Low_Wind_Speed ', 'Aug_Avg_High_Pressure', 
             'Aug_Avg_Pressure', 'Aug_Avg_Low_Pressure', 'Aug_Average_Precipitation', 'Aug_Total_Precipitation', 'Nov_Avg_High_Temperature', 
             'Nov_Avg_Temperature', 'Nov_Avg_Low_Temperature', 'Nov_Avg_High_ Dew_Point', 'Nov_Avg_ Dew_Point', 'Nov_Avg_Low_ Dew_Point', 
             'Nov_Avg_High_Humidity', 'Nov_Avg_Humidity', 'Nov_Avg_Low_Humidity', 'Nov_Avg_High_Wind_Speed ', 'Nov_Avg_Wind_Speed ',
            'Nov_Avg_Low_Wind_Speed ', 'Nov_Avg_High_Pressure', 'Nov_Avg_Pressure', 'Nov_Avg_Low_Pressure', 'Nov_Average_Precipitation', 
            'Nov_Total_Precipitation', 'Jun_Avg_High_Temperature', 'Jun_Avg_Temperature', 'Jun_Avg_Low_Temperature', 'Jun_Avg_High_ Dew_Point', 
            'Jun_Avg_ Dew_Point', 'Jun_Avg_Low_ Dew_Point', 'Jun_Avg_High_Humidity', 'Jun_Avg_Humidity', 'Jun_Avg_Low_Humidity', 'Jun_Avg_High_Wind_Speed ', 
            'Jun_Avg_Wind_Speed ', 'Jun_Avg_Low_Wind_Speed ', 'Jun_Avg_High_Pressure', 'Jun_Avg_Pressure', 'Jun_Avg_Low_Pressure', 'Jun_Average_Precipitation', 
            'Jun_Total_Precipitation', 'Apr_Avg_High_Temperature', 'Apr_Avg_Temperature', 'Apr_Avg_Low_Temperature', 'Apr_Avg_High_ Dew_Point', 'Apr_Avg_ Dew_Point', 
            'Apr_Avg_Low_ Dew_Point', 'Apr_Avg_High_Humidity', 'Apr_Avg_Humidity', 'Apr_Avg_Low_Humidity', 'Apr_Avg_High_Wind_Speed ', 'Apr_Avg_Wind_Speed ', 
            'Apr_Avg_Low_Wind_Speed ', 'Apr_Avg_High_Pressure', 'Apr_Avg_Pressure', 'Apr_Avg_Low_Pressure', 'Apr_Average_Precipitation', 
            'Apr_Total_Precipitation', 'Feb_Avg_High_Temperature', 'Feb_Avg_Temperature', 'Feb_Avg_Low_Temperature', 'Feb_Avg_High_ Dew_Point', 
            'Feb_Avg_ Dew_Point', 'Feb_Avg_Low_ Dew_Point', 'Feb_Avg_High_Humidity', 'Feb_Avg_Humidity', 'Feb_Avg_Low_Humidity', 'Feb_Avg_High_Wind_Speed ', 
            'Feb_Avg_Wind_Speed ', 'Feb_Avg_Low_Wind_Speed ', 'Feb_Avg_High_Pressure', 'Feb_Avg_Pressure', 'Feb_Avg_Low_Pressure', 'Feb_Average_Precipitation', 
            'Feb_Total_Precipitation', 'Oct_Avg_High_Temperature', 'Oct_Avg_Temperature', 'Oct_Avg_Low_Temperature', 'Oct_Avg_High_ Dew_Point', 'Oct_Avg_ Dew_Point', 
            'Oct_Avg_Low_ Dew_Point', 'Oct_Avg_High_Humidity', 'Oct_Avg_Humidity', 'Oct_Avg_Low_Humidity', 'Oct_Avg_High_Wind_Speed ', 'Oct_Avg_Wind_Speed ', 
            'Oct_Avg_Low_Wind_Speed ', 'Oct_Avg_High_Pressure', 'Oct_Avg_Pressure', 'Oct_Avg_Low_Pressure', 'Oct_Average_Precipitation', 'Oct_Total_Precipitation', 
            'Jul_Avg_High_Temperature', 'Jul_Avg_Temperature', 'Jul_Avg_Low_Temperature', 'Jul_Avg_High_ Dew_Point', 'Jul_Avg_ Dew_Point', 'Jul_Avg_Low_ Dew_Point', 
            'Jul_Avg_High_Humidity', 'Jul_Avg_Humidity', 'Jul_Avg_Low_Humidity', 'Jul_Avg_High_Wind_Speed ', 'Jul_Avg_Wind_Speed ', 'Jul_Avg_Low_Wind_Speed ', 
            'Jul_Avg_High_Pressure', 'Jul_Avg_Pressure', 'Jul_Avg_Low_Pressure', 'Jul_Average_Precipitation', 'Jul_Total_Precipitation', 'May_Avg_High_Temperature', 
            'May_Avg_Temperature', 'May_Avg_Low_Temperature', 'May_Avg_High_ Dew_Point', 'May_Avg_ Dew_Point', 'May_Avg_Low_ Dew_Point', 'May_Avg_High_Humidity', 
            'May_Avg_Humidity', 'May_Avg_Low_Humidity', 'May_Avg_High_Wind_Speed ', 'May_Avg_Wind_Speed ', 'May_Avg_Low_Wind_Speed ', 'May_Avg_High_Pressure', 
            'May_Avg_Pressure', 'May_Avg_Low_Pressure', 'May_Average_Precipitation', 'May_Total_Precipitation', 'Dec_Avg_High_Temperature', 'Dec_Avg_Temperature', 
            'Dec_Avg_Low_Temperature', 'Dec_Avg_High_ Dew_Point', 'Dec_Avg_ Dew_Point', 'Dec_Avg_Low_ Dew_Point', 'Dec_Avg_High_Humidity', 'Dec_Avg_Humidity', 
            'Dec_Avg_Low_Humidity', 'Dec_Avg_High_Wind_Speed ', 'Dec_Avg_Wind_Speed ', 'Dec_Avg_Low_Wind_Speed ', 'Dec_Avg_High_Pressure', 'Dec_Avg_Pressure', 
            'Dec_Avg_Low_Pressure', 'Dec_Average_Precipitation', 'Dec_Total_Precipitation', 'Mar_Avg_High_Temperature', 'Mar_Avg_Temperature', 'Mar_Avg_Low_Temperature', 
            'Mar_Avg_High_ Dew_Point', 'Mar_Avg_ Dew_Point', 'Mar_Avg_Low_ Dew_Point', 'Mar_Avg_High_Humidity', 'Mar_Avg_Humidity', 'Mar_Avg_Low_Humidity', 'Mar_Avg_High_Wind_Speed ', 
            'Mar_Avg_Wind_Speed ', 'Mar_Avg_Low_Wind_Speed ', 'Mar_Avg_High_Pressure', 'Mar_Avg_Pressure', 'Mar_Avg_Low_Pressure', 'Mar_Average_Precipitation', 'Mar_Total_Precipitation', 
            'Sep_Avg_High_Temperature', 'Sep_Avg_Temperature', 'Sep_Avg_Low_Temperature', 'Sep_Avg_High_ Dew_Point', 'Sep_Avg_ Dew_Point', 'Sep_Avg_Low_ Dew_Point', 'Sep_Avg_High_Humidity', 
            'Sep_Avg_Humidity', 'Sep_Avg_Low_Humidity', 'Sep_Avg_High_Wind_Speed ', 'Sep_Avg_Wind_Speed ', 'Sep_Avg_Low_Wind_Speed ', 'Sep_Avg_High_Pressure', 'Sep_Avg_Pressure', 
            'Sep_Avg_Low_Pressure', 'Sep_Average_Precipitation', 'Sep_Total_Precipitation', 'Jan_Avg_High_Temperature', 'Jan_Avg_Temperature', 'Jan_Avg_Low_Temperature', 
            'Jan_Avg_High_ Dew_Point', 'Jan_Avg_ Dew_Point', 'Jan_Avg_Low_ Dew_Point', 'Jan_Avg_High_Humidity', 'Jan_Avg_Humidity', 'Jan_Avg_Low_Humidity', 
            'Jan_Avg_High_Wind_Speed ', 'Jan_Avg_Wind_Speed ', 'Jan_Avg_Low_Wind_Speed ', 'Jan_Avg_High_Pressure', 'Jan_Avg_Pressure', 'Jan_Avg_Low_Pressure', 
            'Jan_Average_Precipitation', 'Jan_Total_Precipitation']

In [12]:
text_data = df_filitered[text_columns]
numeric_data = df_filitered[numeric_columns]

In [13]:
encoder = OneHotEncoder()
encoder.fit(text_data)
text_data_encoded = encoder.transform(text_data).toarray()

In [14]:
# Check the number of columns in the encoded text data and numeric data
num_text_columns = text_data_encoded.shape[1]
num_numeric_columns = len(numeric_columns)

# Pad the text data with zeros if necessary
if num_text_columns < num_numeric_columns:
    num_missing_columns = num_numeric_columns - num_text_columns
    text_data_encoded = np.pad(text_data_encoded, ((0,0),(0,num_missing_columns)), 'constant')

In [15]:
data_encoded = np.concatenate((text_data_encoded, numeric_data), axis=1)

In [16]:
# Convert the result of get_feature_names_out to a list
text_column_names = encoder.get_feature_names_out(text_columns)
text_column_names_list = text_column_names.tolist()

In [17]:
# Create a new dataframe with the column names
column_names = text_column_names_list + numeric_columns
data_encoded_df = pd.DataFrame(data_encoded, columns=column_names)

In [18]:
# Save the encoded data to a CSV file
data_encoded_df.to_csv('data_encoded.csv', index=False)

In [19]:
print(data_encoded_df)

     Product_Name_ Wines EGW Chardonnay  Product_Name_ Wines VGR Pinot Noir   \
0                                    0.0                                 0.0   
1                                    0.0                                 0.0   
2                                    0.0                                 0.0   
3                                    0.0                                 0.0   
4                                    0.0                                 0.0   
...                                  ...                                 ...   
1110                                 0.0                                 0.0   
1111                                 0.0                                 0.0   
1112                                 0.0                                 0.0   
1113                                 0.0                                 0.0   
1114                                 0.0                                 0.0   

     Product_Name_Accendo Cellars Laure

In [20]:
## To see what are the column names in the encoded data set 
# Get the column names
column_names = data_encoded_df.columns.tolist()

print(column_names)

['Product_Name_ Wines EGW Chardonnay ', 'Product_Name_ Wines VGR Pinot Noir ', 'Product_Name_Accendo Cellars Laurea Cabernet Sauvignon ', 'Product_Name_Achaval-Ferrer Finca Bella Vista Malbec ', 'Product_Name_Achaval-Ferrer Finca Mirador Malbec ', 'Product_Name_Adaptation by Odette Cabernet Sauvignon ', 'Product_Name_Addendum Napa Valley Cabernet Sauvignon ', 'Product_Name_Alban Central Coast Viognier ', 'Product_Name_Albert Bichot Beaune Clos des Mouches Premier Cru Blanc Domaine du Pavillon ', 'Product_Name_Albert Bichot Clos de la Roche Grand Cru ', 'Product_Name_Albert Bichot Corton Clos Des Marechaudes Domaine du Pavillon Grand Cru ', 'Product_Name_Albert Bichot Meursault Domaine du Pavillon ', 'Product_Name_Albert Bichot Meursault Les Charmes Premier Cru Domaine du Pavillon ', 'Product_Name_Albert Bichot Morey-St-Denis Les Sorbets Premier Cru ', 'Product_Name_Albert Bichot Pommard Clos des Ursulines Domaine du Pavillon Monopole ', 'Product_Name_Allegrini Amarone ', 'Product_Name_

In [21]:
data_encoded_df.dtypes

Product_Name_ Wines EGW Chardonnay                         object
Product_Name_ Wines VGR Pinot Noir                         object
Product_Name_Accendo Cellars Laurea Cabernet Sauvignon     object
Product_Name_Achaval-Ferrer Finca Bella Vista Malbec       object
Product_Name_Achaval-Ferrer Finca Mirador Malbec           object
                                                            ...  
Jan_Avg_High_Pressure                                      object
Jan_Avg_Pressure                                           object
Jan_Avg_Low_Pressure                                       object
Jan_Average_Precipitation                                  object
Jan_Total_Precipitation                                    object
Length: 1660, dtype: object

In [22]:
# Setting the target variable to critical_avg_rating 
y = data_encoded_df['Critical_Avg_Rating']

In [23]:
# select categorical columns
cat_cols = data_encoded_df.select_dtypes(include=['object']).columns

# convert categorical columns to category datatype
data_encoded_df[cat_cols] = data_encoded_df[cat_cols].astype('category')

In [24]:
#Spliting the data set 80:20 split 
X_train, X_test, y_train, y_test = train_test_split(data_encoded_df, y, test_size=0.2, random_state=42)


##attemp 1 hot encoder only 

In [25]:
#model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [26]:
#model.fit(X_train, y_train)

faild could get the obj to work 

    ##atttemp 2 using dmatix

In [27]:
#dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
#dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [28]:
#params = {'objective': 'reg:squarederror', 'random_state': 42}
#num_rounds = 100
#eval_list = [(dtrain, 'train'), (dtest, 'test')]
#eval_metric = 'rmse'
#model = xgb.train(params, dtrain, num_rounds, eval_list, eval_metric)

TypeError: 'str' object is not callable couldn't fix that error agin

#attemp 3

In [31]:
# set enable_categorical=True for categorical columns
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# define parameters for the XGBoost model
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# train the model
num_rounds = 100
eval_list = [(dtrain, 'train'), (dtest, 'test')]
model = xgb.train(params, dtrain, num_rounds, eval_list, verbose_eval=False)


/Users/georgegarcia/Library/Python/3.9/lib/python/site-packages/xgboost/data.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed[data.columns[i]] = (


In [32]:
# make predictions on the test set
y_pred = model.predict(dtest)

## Seeing the performance 

In [33]:
# calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# calculate root mean squared error
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('MSE:', mse)
print('RMSE:', rmse)

MSE: 0.002657367216601114
RMSE: 0.051549657773850587


### adding cross validation 

In [34]:
# create DMatrix objects
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# define parameters for the XGBoost model
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# perform 5-fold cross-validation
num_rounds = 100
cv_results = xgb.cv(params, dtrain, num_boost_round=num_rounds, nfold=5,
                    metrics=['rmse'], early_stopping_rounds=10)

print(cv_results)

/Users/georgegarcia/Library/Python/3.9/lib/python/site-packages/xgboost/data.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed[data.columns[i]] = (


    train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0         83.584749        0.013902       83.584673       0.063139
1         75.242609        0.012536       75.242528       0.064566
2         67.734029        0.011307       67.733946       0.065855
3         60.975829        0.010205       60.975743       0.067023
4         54.893138        0.009217       54.893046       0.068088
..              ...             ...             ...            ...
95         0.028562        0.000511        0.039473       0.005301
96         0.028389        0.000498        0.039466       0.005269
97         0.028224        0.000506        0.039445       0.005282
98         0.028075        0.000512        0.039428       0.005274
99         0.027935        0.000489        0.039415       0.005259

[100 rows x 4 columns]


it seems like some over fitting is happening becasue test is slightly higher than train

In [35]:
## spliting the data into 70:30 for trining and test/validation 30= 15:15 
#Spliting the data set 70:30 split 
X_train, X_valid, y_train, y_valid = train_test_split(data_encoded_df, y, test_size=0.3, random_state=42)


In [36]:
#spliting the data to 50:50 
X_test, X1_valid, y_test, y1_valid =  train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)

In [37]:
# set enable_categorical=True for categorical columns
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
dvalid = xgb.DMatrix(X1_valid, label=y1_valid, enable_categorical=True)
#  train the model with early stopping
num_rounds = 1000
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

# make predictions on the test set
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
y_pred = model.predict(dtest)

/Users/georgegarcia/Library/Python/3.9/lib/python/site-packages/xgboost/data.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed[data.columns[i]] = (


[0]	train-rmse:83.60874	valid-rmse:83.53760
[1]	train-rmse:75.26330	valid-rmse:75.19201
[2]	train-rmse:67.75184	valid-rmse:67.68040
[3]	train-rmse:60.99116	valid-rmse:60.91954
[4]	train-rmse:54.90632	valid-rmse:54.83452
[5]	train-rmse:49.42990	valid-rmse:49.35789
[6]	train-rmse:44.50122	valid-rmse:44.42897
[7]	train-rmse:40.06567	valid-rmse:39.99316
[8]	train-rmse:36.07271	valid-rmse:36.00503
[9]	train-rmse:32.47825	valid-rmse:32.41003
[10]	train-rmse:29.24244	valid-rmse:29.18059
[11]	train-rmse:26.32936	valid-rmse:26.26385
[12]	train-rmse:23.70704	valid-rmse:23.64222
[13]	train-rmse:21.34621	valid-rmse:21.27926
[14]	train-rmse:19.22112	valid-rmse:19.15098
[15]	train-rmse:17.30782	valid-rmse:17.23682
[16]	train-rmse:15.58543	valid-rmse:15.51668
[17]	train-rmse:14.03483	valid-rmse:13.96485
[18]	train-rmse:12.63860	valid-rmse:12.57443
[19]	train-rmse:11.38139	valid-rmse:11.32334
[20]	train-rmse:10.24964	valid-rmse:10.18988
[21]	train-rmse:9.23049	valid-rmse:9.17574
[22]	train-rmse:8.3130

### it seems the overfitting has beeen corrected a bit 

In [38]:
#with  test data set 
# make predictions on the test set
y_pred = model.predict(dtest)

# calculate RMSE on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE on test set:', rmse_test)

RMSE on test set: 0.0387631268356932


##comparing the test vs y_pred

In [46]:
test_df = pd.DataFrame(y_test)

# add a column with the predicted ratings
test_df['predicted_rating'] = y_pred

In [47]:
test_df.head

<bound method NDFrame.head of      Critical_Avg_Rating  predicted_rating
757            93.666667                94
213            92.333333                92
1045               92.25                92
534                 93.6                94
247                 92.6                93
...                  ...               ...
872                 92.0                92
367            96.666667                97
887                 93.0                93
72             92.333333                92
554            96.833333                97

[167 rows x 2 columns]>